In [20]:
import os, json
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
from google.genai import types
from xai_sdk import Client
from xai_sdk.chat import user, system
from anthropic import Anthropic
from loguru import logger
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
XAI_API_KEY = os.getenv("XAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if OPENAI_API_KEY:
    logger.info(f"OpenAI API key is set to {OPENAI_API_KEY[:4]}...{OPENAI_API_KEY[-4:]}")
if XAI_API_KEY:
    logger.info(f"XAI API key is set to {XAI_API_KEY[:4]}...{XAI_API_KEY[-4:]}")
if ANTHROPIC_API_KEY:
    logger.info(f"Anthropic API key is set to {ANTHROPIC_API_KEY[:4]}...{ANTHROPIC_API_KEY[-4:]}")
if DEEPSEEK_API_KEY:
    logger.info(f"DeepSeek API key is set to {DEEPSEEK_API_KEY[:4]}...{DEEPSEEK_API_KEY[-4:]}")
if GOOGLE_API_KEY:
    logger.info(f"Google API key is set to {GOOGLE_API_KEY[:4]}...{GOOGLE_API_KEY[-4:]}")

2025-08-08 23:05:35.080 | INFO     | __main__:<module>:8 - OpenAI API key is set to sk-p...i9cA
2025-08-08 23:05:35.081 | INFO     | __main__:<module>:10 - XAI API key is set to xai-...DhCl
2025-08-08 23:05:35.081 | INFO     | __main__:<module>:12 - Anthropic API key is set to sk-a...fgAA
2025-08-08 23:05:35.081 | INFO     | __main__:<module>:14 - DeepSeek API key is set to sk-9...775f
2025-08-08 23:05:35.081 | INFO     | __main__:<module>:16 - Google API key is set to AIza...Su5Q


In [3]:
request = "What is the meaning of life, the universe, and everything?"
request += " Please answer in a concise and clear manner."
messages = [
    {"role": "user", "content": request}
]

In [4]:
messages

[{'role': 'user',
  'content': 'What is the meaning of life, the universe, and everything? Please answer in a concise and clear manner.'}]

In [5]:
competitors = []
answers = []
messages = [
    {"role": "user", "content": request}
]

In [6]:
openai_client = OpenAI()
response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)
display(Markdown(f"### OpenAI Response:\n{response.choices[0].message.content}"))
competitors.append("OpenAI")
answers.append(response.choices[0].message.content)

### OpenAI Response:
The meaning of life, the universe, and everything is famously humorously summarized as "42," originating from Douglas Adams' science fiction series "The Hitchhiker's Guide to the Galaxy." In a broader philosophical context, it suggests that the search for meaning is complex and subjective, with each individual finding their own purpose through experiences, relationships, and personal beliefs.

In [7]:
claude_client = Anthropic(api_key=ANTHROPIC_API_KEY)
response = claude_client.messages.create(
    model='claude-3-5-haiku-latest',
    messages=messages,
    max_tokens=1000,
)
display(Markdown(f"### Anthropic Response:\n{response.content[0].text}"))
competitors.append("Anthropic")
answers.append(response.content[0].text)

### Anthropic Response:
According to "The Hitchhiker's Guide to the Galaxy" by Douglas Adams, the answer is simply: 42. But in a more philosophical sense, the meaning of life is subjective and can vary based on individual perspectives, which might include personal growth, creating connections, pursuing happiness, or leaving a positive impact on the world.

In [8]:
google_client = genai.Client()

response = google_client.models.generate_content(
    model="gemini-2.5-pro",
    contents=messages[0]['content'],
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=128) # Disables thinking
    ),
)
display(Markdown(f"### Google Gemini Response:\n{response.text}"))
competitors.append("Google Gemini")
answers.append(response.text)

### Google Gemini Response:
This is one of humanity's deepest questions, and it has no single, universally accepted answer. However, we can explore it from a few key perspectives:

**1. The Scientific Perspective:**
From a purely scientific viewpoint, the universe and life don't have an inherent, pre-ordained meaning. Life is a result of complex chemical processes and evolution. The universe operates according to fundamental physical laws. In this sense, "meaning" is not a concept science can measure or discover.

**2. The Philosophical & Personal Perspective:**
This is where most people find their answer. Meaning is not discovered, but *created*. It's what you choose to value and pursue. Common sources of meaning include:

*   **Connection & Love:** Building relationships with family, friends, and community.
*   **Purpose & Contribution:** Making a positive impact on the world, whether through your work, art, volunteering, or simply helping others.
*   **Growth & Experience:** Learning, exploring, overcoming challenges, and appreciating the beauty and wonder of existence.
*   **Happiness & Well-being:** Finding joy, peace, and contentment in the present moment.

**3. The Cultural Reference:**
In Douglas Adams's classic science fiction series, *The Hitchhiker's Guide to the Galaxy*, a supercomputer is built to answer this exact question. After millions of years of calculation, it declares the answer is **42**. The problem, the computer then explains, is that no one knew the *ultimate question* to which 42 was the answer.

---

**In summary:** There is no single answer written in the stars. The meaning of life is the meaning you give it.

In [9]:
deepseek_client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com/v1")
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
)
display(Markdown(f"### DeepSeek Response:\n{response.choices[0].message.content}"))
competitors.append("DeepSeek")
answers.append(response.choices[0].message.content)

### DeepSeek Response:
The meaning of life, the universe, and everything is famously answered as **"42"** in Douglas Adams' *The Hitchhiker's Guide to the Galaxy*—a humorous take on the futility of seeking a single, grand answer.  

Philosophically, the meaning is subjective: it’s shaped by personal purpose, curiosity, and the pursuit of understanding or connection. Science explores *how* the universe exists, while philosophy and spirituality explore *why*.  

In short: **There’s no universal answer—create your own meaning.**

In [10]:
xai_client = Client(
    api_key=os.getenv("XAI_API_KEY"),
    timeout=3600,
)

chat = xai_client.chat.create(model="grok-3-mini")
chat.append(system("You are Grok, a highly intelligent, helpful AI assistant."))
chat.append(user(messages[0]['content']))

response = chat.sample()
display(Markdown(f"### XAI Grok Response:\n{response.content}"))
competitors.append("XAI Grok")
answers.append(response.content)

### XAI Grok Response:
Ah, the ultimate question! As per Douglas Adams' "The Hitchhiker's Guide to the Galaxy," the meaning of life, the universe, and everything is simply **42**. Of course, it's a humorous nod to the idea that some questions might be more profound than their answers suggest. What do you think? 😊

In [11]:
ollama_client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
response = ollama_client.chat.completions.create(
    model="gemma3:12b",
    messages=messages
)
display(Markdown(f"### Ollama Gemma3 Response:\n{response.choices[0].message.content}"))
competitors.append("Ollama Gemma3")
answers.append(response.choices[0].message.content)

### Ollama Gemma3 Response:
According to Douglas Adams' *The Hitchhiker's Guide to the Galaxy*, the answer is **42**.

However, the joke is that nobody knows *what the question is*. The number itself is meaningless without context.



Essentially, the book suggests the search for a definitive meaning is itself the meaning.

In [12]:
print(competitors)
print(answers)

['OpenAI', 'Anthropic', 'Google Gemini', 'DeepSeek', 'XAI Grok', 'Ollama Gemma3']
['The meaning of life, the universe, and everything is famously humorously summarized as "42," originating from Douglas Adams\' science fiction series "The Hitchhiker\'s Guide to the Galaxy." In a broader philosophical context, it suggests that the search for meaning is complex and subjective, with each individual finding their own purpose through experiences, relationships, and personal beliefs.', 'According to "The Hitchhiker\'s Guide to the Galaxy" by Douglas Adams, the answer is simply: 42. But in a more philosophical sense, the meaning of life is subjective and can vary based on individual perspectives, which might include personal growth, creating connections, pursuing happiness, or leaving a positive impact on the world.', 'This is one of humanity\'s deepest questions, and it has no single, universally accepted answer. However, we can explore it from a few key perspectives:\n\n**1. The Scientific P

In [13]:
together = ""
for competitor, answer in zip(competitors, answers):
    together += f"### {competitor} Response:\n{answer}\n\n"
display(Markdown(together))

### OpenAI Response:
The meaning of life, the universe, and everything is famously humorously summarized as "42," originating from Douglas Adams' science fiction series "The Hitchhiker's Guide to the Galaxy." In a broader philosophical context, it suggests that the search for meaning is complex and subjective, with each individual finding their own purpose through experiences, relationships, and personal beliefs.

### Anthropic Response:
According to "The Hitchhiker's Guide to the Galaxy" by Douglas Adams, the answer is simply: 42. But in a more philosophical sense, the meaning of life is subjective and can vary based on individual perspectives, which might include personal growth, creating connections, pursuing happiness, or leaving a positive impact on the world.

### Google Gemini Response:
This is one of humanity's deepest questions, and it has no single, universally accepted answer. However, we can explore it from a few key perspectives:

**1. The Scientific Perspective:**
From a purely scientific viewpoint, the universe and life don't have an inherent, pre-ordained meaning. Life is a result of complex chemical processes and evolution. The universe operates according to fundamental physical laws. In this sense, "meaning" is not a concept science can measure or discover.

**2. The Philosophical & Personal Perspective:**
This is where most people find their answer. Meaning is not discovered, but *created*. It's what you choose to value and pursue. Common sources of meaning include:

*   **Connection & Love:** Building relationships with family, friends, and community.
*   **Purpose & Contribution:** Making a positive impact on the world, whether through your work, art, volunteering, or simply helping others.
*   **Growth & Experience:** Learning, exploring, overcoming challenges, and appreciating the beauty and wonder of existence.
*   **Happiness & Well-being:** Finding joy, peace, and contentment in the present moment.

**3. The Cultural Reference:**
In Douglas Adams's classic science fiction series, *The Hitchhiker's Guide to the Galaxy*, a supercomputer is built to answer this exact question. After millions of years of calculation, it declares the answer is **42**. The problem, the computer then explains, is that no one knew the *ultimate question* to which 42 was the answer.

---

**In summary:** There is no single answer written in the stars. The meaning of life is the meaning you give it.

### DeepSeek Response:
The meaning of life, the universe, and everything is famously answered as **"42"** in Douglas Adams' *The Hitchhiker's Guide to the Galaxy*—a humorous take on the futility of seeking a single, grand answer.  

Philosophically, the meaning is subjective: it’s shaped by personal purpose, curiosity, and the pursuit of understanding or connection. Science explores *how* the universe exists, while philosophy and spirituality explore *why*.  

In short: **There’s no universal answer—create your own meaning.**

### XAI Grok Response:
Ah, the ultimate question! As per Douglas Adams' "The Hitchhiker's Guide to the Galaxy," the meaning of life, the universe, and everything is simply **42**. Of course, it's a humorous nod to the idea that some questions might be more profound than their answers suggest. What do you think? 😊

### Ollama Gemma3 Response:
According to Douglas Adams' *The Hitchhiker's Guide to the Galaxy*, the answer is **42**.

However, the joke is that nobody knows *what the question is*. The number itself is meaningless without context.



Essentially, the book suggests the search for a definitive meaning is itself the meaning.



In [15]:
judge = f"""You are a judge a competition between {len(competitors)} competitors.
Each model have given this question: {messages[0]['content']}.
Your task is to evaluate the responses and rank them from best to worst.

Here are the responses from each competitor:
{together}

You will return a JSON object with the following structure:
```json
{{
    "rankings": [
        {{"competitor": "OpenAI", "score": 0}},
        {{"competitor": "Anthropic", "score": 0}},
        {{"competitor": "Google Gemini", "score": 0}},
        {{"competitor": "DeepSeek", "score": 0}},
        {{"competitor": "XAI Grok", "score": 0}},
        {{"competitor": "Ollama Gemma3", "score": 0}}
    ],
    "explanation": "Your explanation of the rankings."
"""

In [17]:
judge_messages = [
    {"role": "user", "content": judge}
]

In [ ]:
response = openai_client.chat.completions.create(
    model="gpt-5-mini",
    messages=judge_messages
)

In [22]:
result = json.loads(response.choices[0].message.content)
ranks = result['rankings']
explanation = result['explanation']

for index, result in enumerate(ranks):
    print(f"{index + 1}. {result['competitor']} - Score: {result['score']}")
print(f"\nExplanation: {explanation}")

1. Anthropic - Score: 6
2. OpenAI - Score: 5
3. DeepSeek - Score: 4
4. Ollama Gemma3 - Score: 3
5. Google Gemini - Score: 2
6. XAI Grok - Score: 1

Explanation: Ranking criteria: adherence to the request for a concise and clear answer, accuracy (notes the cultural reference to '42'), and useful philosophical context. 1) Anthropic (6): Very concise, names '42' and gives a clear, short philosophical alternative—best balance of brevity and substance. 2) OpenAI (5): Nearly as good; clear and slightly more elaborative while remaining concise. 3) DeepSeek (4): Concise and clear, good summary and direct conclusion to 'create your own meaning.' 4) Ollama Gemma3 (3): Very brief and clear, adds a thoughtful point that the search can be the meaning; slightly more terse than higher-ranked entries. 5) Google Gemini (2): Thorough and clear but much longer and more detailed than requested (less concise). 6) XAI Grok (1): Extremely brief and casual; mentions '42' but offers minimal philosophical conte